# Step 1: Import Libraries

In [ ]:

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


# Step 2: Load the dataset

In [ ]:

data = []
labels = []

# Path to dataset
path = '/New Plant Diseases Dataset(Augmented)/train/'
categories = os.listdir(path)  # All folder names (disease names)

# Create mapping of category names to numeric labels
label_dict = {category: idx for idx, category in enumerate(categories)}

for category in categories:
    folder = os.path.join(path, category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_array = cv2.imread(img_path)
        if img_array is not None:
            img_array = cv2.resize(img_array, (64, 64))  # Resize to 64x64
            data.append(img_array)
            labels.append(label_dict[category])

# Convert to numpy arrays
data = np.array(data) / 255.0  # Normalize images
labels = np.array(labels)


# Step 3: Split the data

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# One-hot encode the labels
num_classes = len(categories)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


# Step 4: Build the CNN Model

In [ ]:

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))  # Softmax for multiple classes


# Step 5: Compile the model

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Step 6: Train the model

In [ ]:

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


# Step 7: Evaluate the model

In [ ]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


# Step 8: Plot Training Graph

In [ ]:

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()
